<a href="https://colab.research.google.com/github/fabriciocarraro/CrewAI_Agent-Tech_job_tailored_CV_creator/blob/main/CrewAI_Agent_Tech_job_tailored_CV_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U  crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.7/462.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool("/content/sample_CV.md")
semantic_search_resume = MDXSearchTool(mdx="/content/sample_CV.md")

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.80it/s]


## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in navigating and extracting critical information from job postings is unmatched."
        "Your skills help pinpoint the necessary qualifications and skills sought by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants to help them stand out in the job market",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect and synthesize information from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they resonate perfectly with the job's requirements."
    )
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, project experiences, contributions, interests, and communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, including the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

In [15]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/29ba09c5-5ed3-44f2-80cc-a31f3a5b1ecd',
    'github_url': 'https://github.com/fabriciocarraro',
    'personal_writeup': """Hands-on multilingual professional focused on the strategic development and execution
     of educational products and platforms; with a strong background in IT programming and language learning.
     Graduated from UNICAMP, one of the most prestigious universities in Latin America. Known to be
     a fast and effective learner—can speak nine languages at least at an intermediate level, most learned
     using self-developed methods. Extensive teaching experience.
     Published author in Brazil on the topic of Artificial Intelligence (AI)."""
}

In [16]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [2024-08-29 20:29:22][DEBUG]: == Working Agent: Tech Job Researcher
 [2024-08-29 20:29:22][INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/29ba09c5-5ed3-44f2-80cc-a31f3a5b1ecd) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [2024-08-29 20:29:22][DEBUG]: == Working Agent: Personal Profiler for Engineers
 [2024-08-29 20:29:22][INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/fabriciocarraro) URLs, and personal write-up (Hands-on multilingual professional focused on the strategic development and execution
     of educational products and platforms; with a strong background in IT programming and language learning.
     Graduated from UNICAMP, one of the most prestigious universities in Latin America. Known to be
     a fast and effective learner—can speak nine languages at least at an 

- Display the generated `tailored_resume.md` file.

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Fabrício Carraro

- **Email:** fabriciocarraro@gmail.com
- **Location:** Barcelona, Spain
- **GitHub:** [github.com/fabriciocarraro](https://github.com/fabriciocarraro)

---

## Profile

Multilingual Artificial Intelligence Engineer with a robust background in software development and educational technology. Experienced in designing and implementing AI solutions that enhance user learning experiences. Graduated from UNICAMP, one of Latin America's premier universities. Recognized for rapid learning abilities and proficiency in nine languages, employing innovative self-developed techniques. Passionate about leveraging technology to drive educational outcomes and committed to maintaining high-quality standards in software design and execution.

---

## Selected Highlights

- Authored a [book](https://www.casadocodigo.com.br/products/livro-inteligencia-artificial-chatgpt) on Artificial Intelligence and Prompt Engineering, contributing to the AI community in Brazil.
- Developed a [RAG PDF Search project](https://github.com/google-gemini/gemma-cookbook/blob/main/Gemma/RAG_PDF_Search_in_multiple_documents_on_Colab.ipynb) featured in Google Gemma's official cookbook, showcasing expertise in AI applications.
- Host of two insightful podcasts: “[IA Sob Controle](https://open.spotify.com/show/5xLCMHJ6eGWzdu8JaIDkuP?si=138eae50eac44b41)” focusing on AI and Machine Learning, and “[Dev Sem Fronteiras](https://www.devsemfronteiras.tech/)” interviewing expatriates.
- Created the "[TechGuide](https://techguide.sh/)" platform to assist individuals in navigating new IT technologies and career development.
- Launched two educational games, [Country Guesser](https://play.google.com/store/apps/details?id=com.fabriciocarraro.countryguesser) and [Language Guesser](https://play.google.com/store/apps/details?id=com.fabriciocarraro.languageguesser), enhancing engagement through gamification on Google Play Store.

---

## Core Competencies

- Artificial Intelligence & Machine Learning
- Software Development & Engineering
- Educational Technology
- Product Management
- Frontend Development (React, Next.js)
- Cloud Services (AWS)
- NoSQL Databases (MongoDB)
- Digital Strategy & Analytics

---

## Education

**Faculdade de Informática e Administração Paulista (FIAP)**  
Postgraduate Course – Data Analytics & Machine Learning | São Paulo, Brazil

**State University of Campinas (UNICAMP)**  
Bachelor of Engineering – Computer Engineering | Campinas, Brazil

---

## Professional Experience

### Alura  
**Program Manager** | Jan 2022 – Present | São Paulo, Brazil (Remote)  
- Spearheaded projects enhancing AI-driven educational tools, including autograding systems and personalized learning experiences.
- Led the "Artificial Intelligence Immersion Week" in collaboration with Google, engaging over 300,000 attendees.
- Established and managed the "[7 Days of Code](https://7daysofcode.io/)" platform, fostering a coding community with over 15k users.
- Integrated strategic initiatives across Alura and its subsidiaries, ensuring cohesive content and product development.

### Alura Língua  
**Content Manager and Business Consultant** | May 2017 – Dec 2021 | São Paulo, Brazil (Remote)  
- Crafted comprehensive content for language courses, enhancing educational offerings and user engagement.
- Optimized SEO strategies, resulting in a 50% increase in website impressions.
- Drove social media engagement, boosting the company's Facebook likes by 70% and achieving a reach of 2.2 million users.

### Crealytics  
**Junior Digital Marketing Manager** | Apr 2016 – Apr 2017 | Passau, Germany  
- Managed and optimized PPC campaigns, leveraging data analytics for performance improvements.
- Automated repetitive tasks using Excel macros in VBA, increasing operational efficiency.

### UOL – PagSeguro  
**Systems Analyst** | Mar 2013 – Apr 2015 | São Paulo, Brazil  
- Developed web applications in Java, focusing on scalability and performance.
- Created a transaction analysis system that improved operational effectiveness by 90%.
- Collaborated with external vendors to integrate third-party fraud solutions, enhancing system security.

### Griaule Biometrics  
**Software Developer** | Aug 2012 – Feb 2013 | Campinas, Brazil  
- Designed and implemented biometric software using C++ and C#.NET, ensuring seamless integration with hardware devices.

---

## Languages

- **Portuguese:** Native
- **Spanish:** Advanced
- **Russian:** Advanced
- **English:** Advanced
- **German:** Advanced
- **Polish:** Advanced
- **Italian:** Advanced
- **French:** Intermediate
- **Greek:** Pre-intermediate

---

## Courses and Certificates

- Advanced Machine Learning, Alura: 2024
- Machine Learning, Alura: 2023
- Python for Data Science, Alura: 2023
- Flutter Development, Alura: 2023
- Front-end Development, Alura: 2022
- Deutsch Als Fremdsprache Level C1, Volkshochschule Passau: 2018
- Advanced Java EE and Web Services, Caelum: 2014
- FCE (First Certificate in English) Level B2, University of Cambridge: 2006

---

## Additional Credentials

- **Technical Skills:**  
  Proficient in Claude, ChatGPT, Gemini, Midjourney, DALL-E, Microsoft Office, Google AdWords, Google Analytics, Flutter, JavaScript, Python, Pandas, C#, Java, Data Structures, SQL, MySQL, Scrum, Kanban, HTML, CSS, Facebook Insights, SEO.
  
- **Other Activities and Projects:**  
  - Founder of language learning podcasts: "Russo para Brasileiros” and “Time to Learn Portuguese."
  - Co-owner of "Language Boost," creating innovative language learning courses.
  - Participated in the "Work & Travel" exchange program in Richmond (VA), USA.
  - Served as Associate Financial Director for the Computer Science and Engineering Academic Athletic Association (AAACEC).
  
- **Hobbies & Interests:**  
  Enthusiastic about traveling, foreign languages, historical fiction and sci-fi literature, podcasting, contemporary cinema, football, and gaming.

- Display the generated `interview_materials.md` file.

In [18]:
display(Markdown("./interview_materials.md"))

```markdown
# Interview Preparation for Fabrício Carraro - Artificial Intelligence Engineer Position at Kira Learning

### Potential Interview Questions and Talking Points 

#### 1. Experience and Background
- **Question:** Can you describe your journey in AI and software development? What motivated you to focus on educational technology?
  - **Talking Point:** Fabrício has a strong educational background with a Bachelor’s in Computer Engineering from UNICAMP and a postgraduate course in Data Analytics & Machine Learning. His experience as a Program Manager at Alura has allowed him to leverage his skills in developing AI-driven educational tools.

#### 2. AI Implementation in Education
- **Question:** How have you utilized AI to enhance the learning experience in your previous roles?
  - **Talking Point:** Fabrício led the development of autograding systems and personalized learning experiences, showcasing his ability to translate complex AI concepts into practical educational tools. His published book on AI underlines his expertise in the field.

#### 3. Project Management and Collaboration
- **Question:** Can you discuss your experience in managing cross-functional teams while working on AI projects?
  - **Talking Point:** Fabrício integrated content and strategy across different teams at Alura, ensuring cohesive product development. He has also collaborated with Google during the "Artificial Intelligence Immersion Week," which attracted over 300,000 attendees.

#### 4. Technical Skills
- **Question:** What is your familiarity with machine learning algorithms and NoSQL databases like MongoDB? Can you provide examples of projects where you utilized these technologies?
  - **Talking Point:** Fabrício has experience with machine learning and programming languages such as Python and Java. He should mention specific projects where he implemented algorithms to solve real-world problems.

#### 5. Attention to Detail
- **Question:** How do you ensure the quality and maintainability of the code in your projects?
  - **Talking Point:** Fabrício has emphasized impeccable attention to detail in his previous roles. He should discuss his experience with code reviews and how he ensures documentation is thorough and clear for future developers.

#### 6. User-Centric Design
- **Question:** How do you prioritize the needs of students and teachers when developing educational products?
  - **Talking Point:** Fabrício's experience in developing user-centric educational platforms like the "[7 Days of Code](https://7daysofcode.io/)" and his efforts in optimizing user engagement metrics show his commitment to understanding user needs.

#### 7. Frontend Development Skills
- **Question:** Can you describe your experience with frontend frameworks like React and Next.js?
  - **Talking Point:** Fabrício should highlight his recent coursework in front-end development and any relevant projects that utilized these frameworks, aligning with the preferred qualifications for the role.

#### 8. Adaptability and Rapid Learning
- **Question:** Can you give an example of a time when you had to quickly adapt to a new technology or project requirement?
  - **Talking Point:** Fabrício is known for being a fast learner and should provide an example from his career, such as mastering a new programming language or adapting to changes in a project scope.

#### 9. Passion for Education and Technology
- **Question:** Why are you passionate about using technology in education, and how do you see the future of AI in this space?
  - **Talking Point:** Fabrício's background as a multilingual educator and content creator showcases his dedication to improving learning experiences. He can discuss trends he observes in AI-driven education.

#### 10. Company Values
- **Question:** Kira Learning values diversity and inclusion. How do you promote these values in your work?
  - **Talking Point:** Fabrício’s multilingual capabilities and international experience could contribute to a diverse workplace. He can also discuss how his podcasts and projects reflect inclusive practices in technology and education.
```